In [1]:
import tensorflow as tf
import numpy as np
import cv2
from keras.utils import Sequence
from keras.applications.mobilenet_v3 import preprocess_input
from glob import glob
import os
from sklearn.model_selection import train_test_split

from keras.applications import MobileNetV3Large
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.models import Model
from keras.optimizers import AdamW
from keras.losses import CategoricalCrossentropy
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.metrics import TopKCategoricalAccuracy
from keras.layers import RandomRotation
from keras.callbacks import LearningRateScheduler



import math
import json


import re
print(tf.__version__)
import tensorflow_probability as tfp
tfd = tfp.distributions


2.15.0




In [2]:
# Load class names
with open("food-101/meta/classes.txt", "r") as file:
    classes = [line.strip() for line in file]

# Read image paths
with open("food-101/meta/train.txt", "r") as file:
    train_paths = [line.strip() for line in file]

with open("food-101/meta/test.txt", "r") as file:
    test_paths = [line.strip() for line in file]

# Helper function to convert label format
def to_snake_case(name):
    return name.replace(" ", "_").lower()

# Convert paths to full paths
train_full_paths = [os.path.join("food-101/images", p + ".jpg") for p in train_paths]
train_labels = [to_snake_case(p.split("/")[0]) for p in train_paths]

test_full_paths = [os.path.join("food-101/images", p + ".jpg") for p in test_paths]
test_labels = [to_snake_case(p.split("/")[0]) for p in test_paths]

# Map labels to indices
class_to_index = {cls: idx for idx, cls in enumerate(classes)}
train_label_indices = [class_to_index[label] for label in train_labels]
test_label_indices = [class_to_index[label] for label in test_labels]

# ✂️ Split 80% train, 20% val
train_paths_split, val_paths_split, train_labels_split, val_labels_split = train_test_split(
    train_full_paths, train_label_indices, test_size=0.2, stratify=train_label_indices, random_state=42
)

# Constants
IMG_SIZE = 224
BATCH_SIZE = 64
NUM_CLASSES = len(classes)

rotation_layer = RandomRotation(factor=0.014)

# Augmentation function
def augment_image(img):
    img = rotation_layer(img)
    img = tf.image.resize_with_crop_or_pad(img, 256, 256)
    img = tf.image.random_crop(img, size=[224, 224, 3])    
    img = tf.image.random_flip_left_right(img)            
    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_contrast(img, 0.8, 1.2)
    img = tf.image.random_saturation(img, 0.9, 1.1)
    img = tf.image.random_hue(img, 0.08)
    img = tf.clip_by_value(img, 0.0, 255.0)
    return img

# Preprocess each sample
def process_example(path, label, augment=False):
    img_raw = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img_raw, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    if augment:
        img = augment_image(img)
    img = preprocess_input(img)
    label = tf.one_hot(label, NUM_CLASSES)
    return img, label

# Dataset creation function
def create_dataset(paths, labels, augment=False, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(paths))
    ds = ds.map(lambda x, y: process_example(x, y, augment), num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

val_ds = create_dataset(val_paths_split, val_labels_split, augment=False, shuffle=False)
test_ds = create_dataset(test_full_paths, test_label_indices, shuffle=False)

In [3]:
def get_cosine_annealing_with_restarts(
    initial_lr=1e-3,
    min_lr=1e-7,
    warmup_epochs=5,
    T_0=20,
    T_mult=1
):
    def scheduler(epoch, lr):

        # Warmup phase
        if epoch < warmup_epochs:
            return float(initial_lr * (epoch + 1) / warmup_epochs)

        # Cosine annealing with restarts
        t = epoch - warmup_epochs  # Epochs after warmup
        T_i = T_0
        cumulative = 0

        while t >= T_i:
            t -= T_i
            cumulative += T_i
            T_i *= T_mult

        progress = t / T_i
        cosine_decay = 0.5 * (1 + math.cos(math.pi * progress))
        return float(min_lr + (initial_lr - min_lr) * cosine_decay)

    return scheduler

In [4]:

def cutmix(images, labels, alpha):
    batch_size = tf.shape(images)[0]
    indices = tf.random.shuffle(tf.range(batch_size))
    image1, image2 = images, tf.gather(images, indices)
    label1, label2 = labels, tf.gather(labels, indices)

    h = tf.shape(images)[1]
    w = tf.shape(images)[2]
    lam = tfd.Beta(alpha, alpha).sample()

    cut_rat = tf.math.sqrt(1. - lam)
    cut_w = tf.cast(tf.cast(w, tf.float32) * cut_rat, tf.int32)
    cut_h = tf.cast(tf.cast(h, tf.float32) * cut_rat, tf.int32)

    cx = tf.random.uniform([], 0, w, dtype=tf.int32)
    cy = tf.random.uniform([], 0, h, dtype=tf.int32)

    x1 = tf.clip_by_value(cx - cut_w // 2, 0, w)
    y1 = tf.clip_by_value(cy - cut_h // 2, 0, h)
    x2 = tf.clip_by_value(cx + cut_w // 2, 0, w)
    y2 = tf.clip_by_value(cy + cut_h // 2, 0, h)

    def replace_patch(img1, img2):
        patch = img2[y1:y2, x1:x2, :]
        paddings = [[y1, h - y2], [x1, w - x2], [0, 0]]
        patch_padded = tf.pad(patch, paddings)
        mask = tf.pad(tf.ones_like(patch), paddings)
        return tf.where(mask > 0, patch_padded, img1)

    mixed_images = tf.map_fn(lambda x: replace_patch(x[0], x[1]), (image1, image2), dtype=tf.float32)

    patch_area = tf.cast((x2 - x1) * (y2 - y1), tf.float32)
    total_area = tf.cast(h * w, tf.float32)
    lam_adjusted = 1.0 - (patch_area / total_area)

    mixed_labels = lam_adjusted * label1 + (1.0 - lam_adjusted) * label2

    return mixed_images, mixed_labels


def mixup(images, labels, alpha):
    batch_size = tf.shape(images)[0]
    indices = tf.random.shuffle(tf.range(batch_size))
    image1, image2 = images, tf.gather(images, indices)
    label1, label2 = labels, tf.gather(labels, indices)

    lam = tfd.Beta(alpha, alpha).sample()
    lam_x = tf.reshape(lam, [1, 1, 1])
    lam_y = tf.reshape(lam, [1])

    mixed_image = lam_x * image1 + (1 - lam_x) * image2
    mixed_label = lam_y * label1 + (1 - lam_y) * label2

    return mixed_image, mixed_label


In [5]:
def apply_mixup_cutmix(ds, alpha_mixup=0.5, alpha_cutmix=1.0):
    counter = tf.data.experimental.Counter()  # will increment per batch
    mixed_ds = tf.data.Dataset.zip((counter, ds)).map(
        lambda i, data: tf.cond(
            tf.equal(i % 2, 0),
            lambda: mixup(data[0], data[1], alpha=alpha_mixup),   # Even → MixUp
            lambda: cutmix(data[0], data[1], alpha=alpha_cutmix)   # Odd → CutMix
        ),
        num_parallel_calls=tf.data.AUTOTUNE
    )
    return mixed_ds

In [6]:
# No Augmentation
train_ds_prep = create_dataset(train_paths_split, train_labels_split, augment=False)
train_ds = apply_mixup_cutmix(train_ds_prep)

loss_fn = CategoricalCrossentropy(label_smoothing=0.05)


base_model = MobileNetV3Large(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.1)(x)
x = Dense(128, activation='relu')(x)
output = Dense(101, activation='softmax')(x)

checkpoint_cb = ModelCheckpoint(
    filepath='D:\\Skripsi\\MobilNetV3-Food101-Rev-Baseline\\Model_MobileNetV3-4-MixUp5+CutMix.keras',          
    monitor='val_accuracy',            
    mode='max',                         
    save_best_only=True,                            
    verbose=1
)

lr_scheduler = LearningRateScheduler(get_cosine_annealing_with_restarts())

model = Model(inputs=base_model.input, outputs=output)
model.compile(
    optimizer=AdamW(learning_rate=1e-3, weight_decay=1e-5), 
    loss=loss_fn, 
    metrics=['accuracy', TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
)

history = model.fit(
    train_ds, 
    validation_data=val_ds, 
    epochs=5, 
    callbacks=[checkpoint_cb, lr_scheduler]
)

total_layers = len(base_model.layers)

unfreeze_count = int(total_layers * 0.3)

for layer in base_model.layers[-unfreeze_count:]:
    layer.trainable = True

model.compile(
    optimizer=AdamW(learning_rate=1e-3, weight_decay=1e-5), 
    loss=loss_fn, 
    metrics=['accuracy', TopKCategoricalAccuracy(k=5, name='top_5_accuracy')]
)

history_fine_tune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
    initial_epoch=5,
    callbacks=[checkpoint_cb, lr_scheduler]
)

combined_history = {}
for key in history.history.keys():
    combined_history[key] = history.history[key] + history_fine_tune.history[key]

with open('skripsi/history/Model_History_MobilNetV3-4-MixUp5+CutMix.json', 'w') as f:
    json.dump(combined_history, f)

Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.
Instructions for updating:
Use fn_output_signature instead

Epoch 1/5

947/947 [==============================] - ETA: 0s - loss: 3.9303 - accuracy: 0.1845 - top_5_accuracy: 0.3844
Epoch 1: val_accuracy improved from -inf to 0.45320, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 811s 852ms/step - loss: 3.9303 - accuracy: 0.1845 - top_5_accuracy: 0.3844 - val_loss: 2.5364 - val_accuracy: 0.4532 - val_top_5_accuracy: 0.7368 - lr: 2.0000e-04
Epoch 2/5
947/947 [==============================] - ETA: 0s - loss: 3.3266 - accuracy: 0.3430 - top_5_accuracy: 0.6057
Epoch 2: val_accuracy improved from 0.45320 to 0.54647, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 764s 805ms/step - loss: 3.3266 - accuracy: 0.3430 - top_5_accuracy: 0.60

TypeError: Object of type float32 is not JSON serializable

WARNING:tensorflow:From C:\Users\Lucas Salvacio\AppData\Local\Temp\ipykernel_20384\2703249703.py:2: CounterV2 (from tensorflow.python.data.experimental.ops.counter) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.
WARNING:tensorflow:From c:\Users\Lucas Salvacio\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\util\deprecation.py:660: calling map_fn_v2 (from tensorflow.python.ops.map_fn) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Use fn_output_signature instead
WARNING:tensorflow:From c:\Users\Lucas Salvacio\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\normalization\batch_normalization.py:979: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

Epoch 1/5
WARNING:tensorflow:From c:\Users\Lucas Salvacio\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\tf_utils.py:492: The name tf.ragged.RaggedTensorValue is deprecated. Please use tf.compat.v1.ragged.RaggedTensorValue instead.

947/947 [==============================] - ETA: 0s - loss: 3.9303 - accuracy: 0.1845 - top_5_accuracy: 0.3844
Epoch 1: val_accuracy improved from -inf to 0.45320, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 811s 852ms/step - loss: 3.9303 - accuracy: 0.1845 - top_5_accuracy: 0.3844 - val_loss: 2.5364 - val_accuracy: 0.4532 - val_top_5_accuracy: 0.7368 - lr: 2.0000e-04
Epoch 2/5
947/947 [==============================] - ETA: 0s - loss: 3.3266 - accuracy: 0.3430 - top_5_accuracy: 0.6057
Epoch 2: val_accuracy improved from 0.45320 to 0.54647, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 764s 805ms/step - loss: 3.3266 - accuracy: 0.3430 - top_5_accuracy: 0.6057 - val_loss: 2.1580 - val_accuracy: 0.5465 - val_top_5_accuracy: 0.8059 - lr: 4.0000e-04
Epoch 3/5
947/947 [==============================] - ETA: 0s - loss: 3.1863 - accuracy: 0.3818 - top_5_accuracy: 0.6445
Epoch 3: val_accuracy improved from 0.54647 to 0.57129, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 736s 776ms/step - loss: 3.1863 - accuracy: 0.3818 - top_5_accuracy: 0.6445 - val_loss: 2.0520 - val_accuracy: 0.5713 - val_top_5_accuracy: 0.8205 - lr: 6.0000e-04
Epoch 4/5
947/947 [==============================] - ETA: 0s - loss: 3.1447 - accuracy: 0.3986 - top_5_accuracy: 0.6536
Epoch 4: val_accuracy improved from 0.57129 to 0.58185, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 734s 774ms/step - loss: 3.1447 - accuracy: 0.3986 - top_5_accuracy: 0.6536 - val_loss: 2.0146 - val_accuracy: 0.5818 - val_top_5_accuracy: 0.8305 - lr: 8.0000e-04
Epoch 5/5
947/947 [==============================] - ETA: 0s - loss: 3.1291 - accuracy: 0.4047 - top_5_accuracy: 0.6610
Epoch 5: val_accuracy improved from 0.58185 to 0.58653, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 732s 772ms/step - loss: 3.1291 - accuracy: 0.4047 - top_5_accuracy: 0.6610 - val_loss: 1.9725 - val_accuracy: 0.5865 - val_top_5_accuracy: 0.8359 - lr: 0.0010
Epoch 6/25
947/947 [==============================] - ETA: 0s - loss: 2.9807 - accuracy: 0.4576 - top_5_accuracy: 0.7100
Epoch 6: val_accuracy did not improve from 0.58653
947/947 [==============================] - 925s 971ms/step - loss: 2.9807 - accuracy: 0.4576 - top_5_accuracy: 0.7100 - val_loss: 2.7884 - val_accuracy: 0.4536 - val_top_5_accuracy: 0.7242 - lr: 0.0010
Epoch 7/25
947/947 [==============================] - ETA: 0s - loss: 2.7153 - accuracy: 0.5409 - top_5_accuracy: 0.7781
Epoch 7: val_accuracy did not improve from 0.58653
947/947 [==============================] - 920s 970ms/step - loss: 2.7153 - accuracy: 0.5409 - top_5_accuracy: 0.7781 - val_loss: 2.2100 - val_accuracy: 0.5521 - val_top_5_accuracy: 0.8100 - lr: 9.9384e-04
Epoch 8/25
947/947 [==============================] - ETA: 0s - loss: 2.5860 - accuracy: 0.5827 - top_5_accuracy: 0.8063
Epoch 8: val_accuracy improved from 0.58653 to 0.65129, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 917s 968ms/step - loss: 2.5860 - accuracy: 0.5827 - top_5_accuracy: 0.8063 - val_loss: 1.8338 - val_accuracy: 0.6513 - val_top_5_accuracy: 0.8708 - lr: 9.7553e-04
Epoch 9/25
947/947 [==============================] - ETA: 0s - loss: 2.4659 - accuracy: 0.6149 - top_5_accuracy: 0.8284
Epoch 9: val_accuracy improved from 0.65129 to 0.67617, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 917s 967ms/step - loss: 2.4659 - accuracy: 0.6149 - top_5_accuracy: 0.8284 - val_loss: 1.7258 - val_accuracy: 0.6762 - val_top_5_accuracy: 0.8903 - lr: 9.4551e-04
Epoch 10/25
947/947 [==============================] - ETA: 0s - loss: 2.4549 - accuracy: 0.6177 - top_5_accuracy: 0.8311
Epoch 10: val_accuracy improved from 0.67617 to 0.71122, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 916s 967ms/step - loss: 2.4549 - accuracy: 0.6177 - top_5_accuracy: 0.8311 - val_loss: 1.5674 - val_accuracy: 0.7112 - val_top_5_accuracy: 0.9028 - lr: 9.0452e-04
Epoch 11/25
947/947 [==============================] - ETA: 0s - loss: 2.3160 - accuracy: 0.6581 - top_5_accuracy: 0.8591
Epoch 11: val_accuracy improved from 0.71122 to 0.72389, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 917s 967ms/step - loss: 2.3160 - accuracy: 0.6581 - top_5_accuracy: 0.8591 - val_loss: 1.4891 - val_accuracy: 0.7239 - val_top_5_accuracy: 0.9073 - lr: 8.5357e-04
Epoch 12/25
947/947 [==============================] - ETA: 0s - loss: 2.3144 - accuracy: 0.6624 - top_5_accuracy: 0.8555
Epoch 12: val_accuracy improved from 0.72389 to 0.72713, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 917s 967ms/step - loss: 2.3144 - accuracy: 0.6624 - top_5_accuracy: 0.8555 - val_loss: 1.4821 - val_accuracy: 0.7271 - val_top_5_accuracy: 0.9098 - lr: 7.9391e-04
Epoch 13/25
947/947 [==============================] - ETA: 0s - loss: 2.2603 - accuracy: 0.6807 - top_5_accuracy: 0.8678
Epoch 13: val_accuracy improved from 0.72713 to 0.74139, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 919s 969ms/step - loss: 2.2603 - accuracy: 0.6807 - top_5_accuracy: 0.8678 - val_loss: 1.4457 - val_accuracy: 0.7414 - val_top_5_accuracy: 0.9150 - lr: 7.2702e-04
Epoch 14/25
947/947 [==============================] - ETA: 0s - loss: 2.2031 - accuracy: 0.6952 - top_5_accuracy: 0.8761
Epoch 14: val_accuracy improved from 0.74139 to 0.75380, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 917s 968ms/step - loss: 2.2031 - accuracy: 0.6952 - top_5_accuracy: 0.8761 - val_loss: 1.3787 - val_accuracy: 0.7538 - val_top_5_accuracy: 0.9198 - lr: 6.5454e-04
Epoch 15/25
947/947 [==============================] - ETA: 0s - loss: 2.1423 - accuracy: 0.7157 - top_5_accuracy: 0.8864
Epoch 15: val_accuracy improved from 0.75380 to 0.75980, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 917s 968ms/step - loss: 2.1423 - accuracy: 0.7157 - top_5_accuracy: 0.8864 - val_loss: 1.3561 - val_accuracy: 0.7598 - val_top_5_accuracy: 0.9224 - lr: 5.7826e-04
Epoch 16/25
947/947 [==============================] - ETA: 0s - loss: 2.0950 - accuracy: 0.7256 - top_5_accuracy: 0.8939
Epoch 16: val_accuracy improved from 0.75980 to 0.76462, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 917s 968ms/step - loss: 2.0950 - accuracy: 0.7256 - top_5_accuracy: 0.8939 - val_loss: 1.3466 - val_accuracy: 0.7646 - val_top_5_accuracy: 0.9222 - lr: 5.0005e-04
Epoch 17/25
947/947 [==============================] - ETA: 0s - loss: 2.0192 - accuracy: 0.7532 - top_5_accuracy: 0.9079
Epoch 17: val_accuracy improved from 0.76462 to 0.76680, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 920s 970ms/step - loss: 2.0192 - accuracy: 0.7532 - top_5_accuracy: 0.9079 - val_loss: 1.3111 - val_accuracy: 0.7668 - val_top_5_accuracy: 0.9271 - lr: 4.2184e-04
Epoch 18/25
947/947 [==============================] - ETA: 0s - loss: 2.0339 - accuracy: 0.7533 - top_5_accuracy: 0.9072
Epoch 18: val_accuracy improved from 0.76680 to 0.76898, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 917s 968ms/step - loss: 2.0339 - accuracy: 0.7533 - top_5_accuracy: 0.9072 - val_loss: 1.3001 - val_accuracy: 0.7690 - val_top_5_accuracy: 0.9265 - lr: 3.4556e-04
Epoch 19/25
947/947 [==============================] - ETA: 0s - loss: 1.9548 - accuracy: 0.7610 - top_5_accuracy: 0.9103
Epoch 19: val_accuracy improved from 0.76898 to 0.77360, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 918s 968ms/step - loss: 1.9548 - accuracy: 0.7610 - top_5_accuracy: 0.9103 - val_loss: 1.2853 - val_accuracy: 0.7736 - val_top_5_accuracy: 0.9284 - lr: 2.7308e-04
Epoch 20/25
947/947 [==============================] - ETA: 0s - loss: 1.9619 - accuracy: 0.7695 - top_5_accuracy: 0.9149
Epoch 20: val_accuracy improved from 0.77360 to 0.77439, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 918s 969ms/step - loss: 1.9619 - accuracy: 0.7695 - top_5_accuracy: 0.9149 - val_loss: 1.2789 - val_accuracy: 0.7744 - val_top_5_accuracy: 0.9296 - lr: 2.0619e-04
Epoch 21/25
947/947 [==============================] - ETA: 0s - loss: 1.9324 - accuracy: 0.7764 - top_5_accuracy: 0.9174
Epoch 21: val_accuracy improved from 0.77439 to 0.77558, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 968s 1s/step - loss: 1.9324 - accuracy: 0.7764 - top_5_accuracy: 0.9174 - val_loss: 1.2709 - val_accuracy: 0.7756 - val_top_5_accuracy: 0.9313 - lr: 1.4653e-04
Epoch 22/25
947/947 [==============================] - ETA: 0s - loss: 1.9095 - accuracy: 0.7843 - top_5_accuracy: 0.9190
Epoch 22: val_accuracy improved from 0.77558 to 0.77617, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 979s 1s/step - loss: 1.9095 - accuracy: 0.7843 - top_5_accuracy: 0.9190 - val_loss: 1.2761 - val_accuracy: 0.7762 - val_top_5_accuracy: 0.9302 - lr: 9.5582e-05
Epoch 23/25
947/947 [==============================] - ETA: 0s - loss: 1.8741 - accuracy: 0.7825 - top_5_accuracy: 0.9206
Epoch 23: val_accuracy improved from 0.77617 to 0.77683, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 977s 1s/step - loss: 1.8741 - accuracy: 0.7825 - top_5_accuracy: 0.9206 - val_loss: 1.2636 - val_accuracy: 0.7768 - val_top_5_accuracy: 0.9325 - lr: 5.4591e-05
Epoch 24/25
947/947 [==============================] - ETA: 0s - loss: 1.9252 - accuracy: 0.7786 - top_5_accuracy: 0.9177
Epoch 24: val_accuracy improved from 0.77683 to 0.77789, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 980s 1s/step - loss: 1.9252 - accuracy: 0.7786 - top_5_accuracy: 0.9177 - val_loss: 1.2679 - val_accuracy: 0.7779 - val_top_5_accuracy: 0.9317 - lr: 2.4569e-05
Epoch 25/25
947/947 [==============================] - ETA: 0s - loss: 1.8574 - accuracy: 0.7941 - top_5_accuracy: 0.9256
Epoch 25: val_accuracy improved from 0.77789 to 0.77894, saving model to D:\Skripsi\MobilNetV3-Food101-Rev-Baseline\Model_MobileNetV3-4-MixUp5+CutMix.keras
947/947 [==============================] - 1032s 1s/step - loss: 1.8574 - accuracy: 0.7941 - top_5_accuracy: 0.9256 - val_loss: 1.2633 - val_accuracy: 0.7789 - val_top_5_accuracy: 0.9323 - lr: 6.2552e-06


In [ ]:
results = model.evaluate(test_ds)

395/395 [==============================] - 233s 588ms/step - loss: 1.0629 - accuracy: 0.8313 - top_5_accuracy: 0.9588
